In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
data = pd.read_csv("cardio_train.csv",sep=";")
# print(data.shape)
# print(data.columns)
labels = data.iloc[:,-1]
data = data.iloc[:,1:-1]
X_train, X_test, y_train, y_test = train_test_split(data,labels,test_size=0.1,random_state=42)
X_train["target"]=y_train.to_numpy()


<ipython-input-1-a1134f3f2fb4>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["target"]=y_train.to_numpy()


In [2]:
def entropy(target):
        distinct_Values = target.nunique()
        value_counts = target.value_counts().to_numpy()
        total = value_counts.sum()
        entropy =0
        for term in range(distinct_Values):
            entropy=entropy + value_counts[term]/total * np.log2( value_counts[term]/total)
        return -1*entropy

def entropy_eq(num1,num2):
    denum = num1+num2
    if (num1 ==0) or (num2 ==0):
        return 0 
    else:
        return -1*(num1/denum * np.log2(num1/denum) + num2/denum * np.log2(num2/denum))

In [3]:
def information_gain(feature_target):
    distinct_Values = feature_target.iloc[:,0].nunique()
    value_counts = feature_target.iloc[:,0].value_counts().to_numpy()
    total = value_counts.sum()
    target_zerocounts = feature_target.groupby(feature_target.iloc[:,0])["target"].agg(lambda x: x.eq(0).sum()).to_numpy()
    target_onescounts = feature_target.groupby(feature_target.iloc[:,0])["target"].agg(lambda x: x.eq(1).sum()).to_numpy()
    info_gain = entropy(feature_target.iloc[:,1])
    for value in range(distinct_Values):
        info_gain = info_gain - value_counts[value]/total * entropy_eq(target_zerocounts[value],target_onescounts[value])
    return info_gain
        
def gini_impurity(feature_target):
   counts = feature_target.groupby(feature_target.columns[0])["target"].value_counts().to_numpy()
   propability =  counts/feature_target.shape[0]
   gini = 1 - np.square(propability).sum()

   return gini
info_gain = information_gain(X_train.iloc[:,[6,-1]])
gini = gini_impurity(X_train.iloc[:,[10,-1]])


In [4]:
#Code that calculates gini or info gain for all features and return index of best split
metrictype = "gini"
def calculate_metric(df):
    metric = []
    if metrictype == "gini":
        for feature in range(df.shape[1]-1):
            metric.append(gini_impurity(df.iloc[:,[feature,-1]]))
        print(metric)
        best_splitidx = np.where(metric==min(metric))
    elif metrictype == "ig":
        for feature in range(df.shape[1]-1):
            metric.append(information_gain(X_train.iloc[:,[feature,-1]]))
        print(metric)
        best_splitidx = np.where(metric==(max(metric)))
    return best_splitidx
    
bestfeature = calculate_metric(X_train)

[0.9998955364071554, 0.727043880574452, 0.9793469690098262, 0.9874093837238599, 0.8785581909800957, 0.8308109412950365, 0.6975700256991686, 0.632712052910053, 0.5804446495338876, 0.5511076336608718, 0.657478634920635]


In [10]:
def find_split(bestfeature,df):
    bestcol_name = df.columns[bestfeature[0][0]]
    print(df[bestcol_name].unique())
    df = df.sort_values(bestcol_name)
    value_counts = df.shape[0]
    gini = []
    for value in df.iloc[:,bestfeature[0][0]].unique():
        leftdf = df[df[bestcol_name]<=value]
        rightdf = df[df[bestcol_name]>value]
        # print(value)
        if rightdf.empty:
            break
        else:
            ginileft = gini_impurity(leftdf[[bestcol_name,"target"]])
            giniright =  gini_impurity(rightdf[[bestcol_name,"target"]])
            gini.append(leftdf.shape[0]/value_counts * ginileft + rightdf.shape[0]/value_counts * giniright)
    gini = np.asarray(gini)
    bestsplit_idx = np.where(gini==min(gini))
    values =  df.iloc[:,bestfeature[0][0]].unique()
    splitvalue = values[bestsplit_idx]
    leftnode = df[df[bestcol_name]<=splitvalue]
    rightnode = rightdf = df[df[bestcol_name]>splitvalue]
    

find_split(bestfeature,X_train)


[0 1]
